# Cross over conversation generator
This note book only works on google colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import random
import re


In [ ]:
gpt2.download_gpt2(model_name="124M")

# Introduction of the checkpoints
There are four models available for generating in this notebook.Before generating anything, all of the check point files need to be uploaded into a folder called "checkpoint", and the folder "checkpoint" needs to be created under "content" folder. 

In [5]:
sess1 = gpt2.start_tf_sess()
sess2 = gpt2.start_tf_sess()
sess3 = gpt2.start_tf_sess()
sess4 = gpt2.start_tf_sess()

In [6]:
gpt2.load_gpt2(sess1, run_name='Bilbo')

Loading checkpoint checkpoint/Bilbo/model-100
INFO:tensorflow:Restoring parameters from checkpoint/Bilbo/model-100


In [7]:
tf.get_variable_scope().reuse_variables()
gpt2.load_gpt2(sess2, run_name='Hagrid')

Loading checkpoint checkpoint/Hagrid/model-100
INFO:tensorflow:Restoring parameters from checkpoint/Hagrid/model-100


In [8]:
tf.get_variable_scope().reuse_variables()
gpt2.load_gpt2(sess3, run_name='Gandalf')

Loading checkpoint checkpoint/Gandalf/model-100
INFO:tensorflow:Restoring parameters from checkpoint/Gandalf/model-100


In [9]:
tf.get_variable_scope().reuse_variables()
gpt2.load_gpt2(sess4, run_name='Dumbledore')

Loading checkpoint checkpoint/Dumbledore/model-100
INFO:tensorflow:Restoring parameters from checkpoint/Dumbledore/model-100


# Introduction of 
# function generate_2(model_name1, model_name2, text, max_number)
To make it easier to generate a conversation, I wrote the following function，the idea is to let two models generate prefix for each other. To mantain the consistency within each conversation, i made some effort to unify the format of each generated sentence, in order to avoid generating empty spaces or single punctuation marks.However, my method can not completely eliminate the generation of empty information, but can make the probability of such events relatively smaller.  



In [10]:

def generate_2(model_name1, model_name2, text, max_number):
  text = str(text)
  session = {'Bilbo':sess1, 'Hagrid':sess2, 'Gandalf':sess3, 'Dumbledore':sess4}
  out = {'Bilbo':'[Bilbo out]', 'Hagrid':'[Hagrid out]', 'Gandalf':'[out]', 'Dumbledore':'[out]'}
  max_number = int(max_number)
  conversation = []
  pattern = re.compile(r'\W+$')

  
  while len(conversation)<=max_number:
    if len(conversation)<=0:
        text = gpt2.generate(session[model_name1], 
                             return_as_list=True, 
                             prefix=text, 
                             nsamples=1, 
                             length=30, temperature=0.5, 
                             run_name=model_name1, 
                             truncate =out[model_name1], 
                             include_prefix = True,
                             )
        text = str(text[0])
        text = re.sub(pattern,',',text)
        list = ['0']
        list[0] = text
        conversation.append(list)

    text = gpt2.generate(session[model_name2], 
                     return_as_list=True, 
                     prefix=list[0], 
                     nsamples=1, 
                     length=30, temperature=0.5, 
                     run_name=model_name2, 
                     truncate = out[model_name2], 
                     include_prefix = False,
                     )
    text = str(text[0])
    text = re.sub(pattern,',',text)
    list = ['0']
    list[0] = text
    conversation.append(list)

    text = gpt2.generate(session[model_name1], 
                     return_as_list=True, 
                     prefix=list[0], 
                     nsamples=1, 
                     length=30, temperature=0.5, 
                     run_name=model_name1, 
                     truncate = out[model_name1], 
                     include_prefix = False,
                     )
    text = str(text[0])
    text = re.sub(pattern,',',text)
    list = ['0']
    list[0] = text
    conversation.append(list) 

    

    
  conversation_new = []
  for i in range(len(conversation)):
    line = conversation[i]
    if line == ['']:
      line = '[......]'
    conversation_new.append(line)
  for i in range(len(conversation_new)):
    if i%2 == 0:
      print('[', model_name1, ':]', conversation_new[i])
    elif i%2 != 0:
      print('[', model_name2, ':]', conversation_new[i])

  


In [ ]:
generate_2('Bilbo', 'Hagrid', "I think dragons are", 5)

[ Bilbo :] ['I think dragons are a terrible lot and the worst thing they can do is cause trouble,']
[ Hagrid :] [" even if it's just for show,"]
[ Bilbo :] [' you know,']
[ Hagrid :] [" I think he killed 'im one day"]
[ Bilbo :] [',']
[ Hagrid :] ["Hagrid out]Tha's the one"]
[ Bilbo :] [' thing I never forgave you, oh Nagrand’s Smaug the Stupendous,']


In [ ]:
generate_2('Hagrid','Bilbo', 'I think dragons are', 5)

[ Hagrid :] ['I think dragons are gonna like me']
[ Bilbo :] [' for it,']
[ Hagrid :] [" but I don't think so,"]
[ Bilbo :] [' either,']
[ Hagrid :] [" but I reckon he'll be glad of it,"]
[ Bilbo :] [' for his guard is up,']
[ Hagrid :] [" but I'm not there yet,"]


In [11]:
generate_2('Gandalf', 'Dumbledore', "Good morning", 5)

[ Gandalf :] ['Good morning, all." ―Fili, Dain, Balin, Bifur, Bofur, Bofur-ul-Fili,']
[ Dumbledore :] [" 'The Hall goes dark,"]
[ Gandalf :] [' and the only thing left to do is leave the books in the room,']
[ Dumbledore :] [' so Sirius can return them,']
[ Gandalf :] [' if he so chooses,']
[ Dumbledore :] [" but I'm afraid he'll find it much more challenging than I imagined,"]
[ Gandalf :] [' and he does than which I can only speak Persian,']


In [16]:
generate_2('Dumbledore','Gandalf', "Good morning", 5)

[ Dumbledore :] ['Good morning, Harry,']
[ Gandalf :] [' and good morning to our friends,']
[ Dumbledore :] [' Minerva,']
[ Gandalf :] [' or else it will fall into the wrong hands,']
[ Dumbledore :] [' allowing the wrong person to take it,']
[ Gandalf :] [' and thus the game of golf is abandoned,']
[ Dumbledore :] [' the gamekeeper is more than willing to send the gamekeeper to the dungeons']
